In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import joblib
from joblib import dump
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier, VotingClassifier

In [5]:
data = pd.read_csv("C:/Users/Ramya/Downloads/Capstone_Final_Dataset.csv")

In [6]:
data = data.dropna()

In [7]:
data.shape[0]

548

In [8]:
label_encoder = LabelEncoder()
data['OBESITY'] = label_encoder.fit_transform(data['OBESITY'])
data['MENSTRUAL IRREGULARITY'] = label_encoder.fit_transform(data['MENSTRUAL IRREGULARITY'])
data['THYROID PROBLEM'] = label_encoder.fit_transform(data['THYROID PROBLEM'])
data['CARDIOVASCULAR DISEASE'] = label_encoder.fit_transform(data['CARDIOVASCULAR DISEASE'])
data['PCOS'] = label_encoder.fit_transform(data['PCOS'])
data['HYPERTENSION'] = label_encoder.fit_transform(data['HYPERTENSION'])
data['DIABETES'] = label_encoder.fit_transform(data['DIABETES'])
data['HOT FLASHES'] = label_encoder.fit_transform(data['HOT FLASHES'])
data['SWEAT'] = label_encoder.fit_transform(data['SWEAT'])
data['MOOD SWINGS'] = label_encoder.fit_transform(data['MOOD SWINGS'])
data['FATIGUE'] = label_encoder.fit_transform(data['FATIGUE'])
data['Lifestyle Modification'] = label_encoder.fit_transform(data['Lifestyle Modification'])
data['T2DM'] = label_encoder.fit_transform(data['T2DM'])
data['Less salt intake'] = label_encoder.fit_transform(data['Less salt intake'])
data['Regular exercise'] = label_encoder.fit_transform(data['Regular exercise'])
data['Nutrition rich food'] = label_encoder.fit_transform(data['Nutrition rich food'])
data['Medication'] = label_encoder.fit_transform(data['Medication'])
data['SEVERITY'] = label_encoder.fit_transform(data['SEVERITY'])

In [9]:
thresholds = {
    'HB': (12, 16),
    'SBP': (90, 120),
    'DBP': (60, 80),
    'FSH': (25.8, 134.8),
    'PROLACTIN': (2, 25),
    'ESTROGEN': (30, 400),
    'LH': (14.2, 52.3),
    'PROGESTERONE': (0.2, 1),
    'GNRH': (2, 10),
    'VITAMIN D': (20, 40),
    'TSH': (0.46, 4),
    'FT3': (2.3, 4.1),
    'FT4': (0.8, 1.8)
}

for feature, (low, high) in thresholds.items():
    data[feature + '_Category'] = pd.cut(data[feature], bins=[-float('inf'), low, high, float('inf')], labels=['Low', 'Normal', 'High'])

In [10]:
x = data[['AGE','OBESITY','MENSTRUAL IRREGULARITY','THYROID PROBLEM','CARDIOVASCULAR DISEASE','PCOS','HYPERTENSION','DIABETES','HEIGHT','WEIGHT','BMI','HB','FSH','ESTROGEN','PROGESTERONE','VITAMIN D','FT3','HEART RATE','SBP','DBP','PROLACTIN','LH','GNRH','TSH','FT4','HDL','TG','HOT FLASHES','SWEAT','MOOD SWINGS','FATIGUE']]
y = data['SEVERITY']

k_best = 8
selector = SelectKBest(score_func=chi2, k=k_best)
x_train_select = selector.fit_transform(x,y)

selected_features = x.columns[selector.get_support()]
print("Selected Features:", selected_features)

Selected Features: Index(['AGE', 'MENSTRUAL IRREGULARITY', 'THYROID PROBLEM', 'WEIGHT',
       'ESTROGEN', 'FT3', 'HEART RATE', 'LH'],
      dtype='object')


In [11]:
X = data[['AGE', 'MENSTRUAL IRREGULARITY', 'THYROID PROBLEM', 'WEIGHT',
       'ESTROGEN', 'FT3', 'HEART RATE', 'LH','MOOD SWINGS','HOT FLASHES']]
recommendations = ['Lifestyle Modification', 'T2DM', 'Less salt intake', 'Regular exercise', 'Nutrition rich food','Medication']
rec = data[recommendations]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [13]:
X_train_rec, X_test_rec, y_train_rec, y_test_rec = train_test_split(X, rec, test_size=0.2, random_state=42)

In [14]:
# MLPClassifier
mlp_model = MLPClassifier()
mlp_param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (150,)],
    'activation': ['relu', 'logistic', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01]
}
mlp_grid_search = GridSearchCV(mlp_model, mlp_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
mlp_grid_search.fit(X_train, y_train)
mlp_best_model = mlp_grid_search.best_estimator_

c:\Users\Ramya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [20]:
# ExtraTreesClassifier
extra_trees_model = ExtraTreesClassifier()
extra_trees_param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
extra_trees_grid_search = GridSearchCV(extra_trees_model, extra_trees_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
extra_trees_grid_search.fit(X_train, y_train)
extra_trees_best_model = extra_trees_grid_search.best_estimator_

In [21]:
# RandomForestClassifier
rf_model = RandomForestClassifier()
rf_param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
rf_grid_search.fit(X_train, y_train)
rf_best_model = rf_grid_search.best_estimator_

In [22]:
# GradientBoostingClassifier
gb_model = GradientBoostingClassifier()
gb_param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7]
}
gb_grid_search = GridSearchCV(gb_model, gb_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
gb_grid_search.fit(X_train, y_train)
gb_best_model = gb_grid_search.best_estimator_

In [23]:
# AdaBoostClassifier
ada_model = AdaBoostClassifier()
ada_param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.5]
}
ada_grid_search = GridSearchCV(ada_model, ada_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
ada_grid_search.fit(X_train, y_train)
ada_best_model = ada_grid_search.best_estimator_

c:\Users\Ramya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [24]:
# XGBClassifier
xgb_model = XGBClassifier()
xgb_param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7]
}
xgb_grid_search = GridSearchCV(xgb_model, xgb_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
xgb_grid_search.fit(X_train, y_train)
xgb_best_model = xgb_grid_search.best_estimator_

In [25]:
# KNeighborsClassifier
knn_model = KNeighborsClassifier()
knn_param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree']
}
knn_grid_search = GridSearchCV(knn_model, knn_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
knn_grid_search.fit(X_train, y_train)
knn_best_model = knn_grid_search.best_estimator_

In [26]:
# Generate predictions from base models
svc_model = SVC()
svc_model.fit(X_train,y_train)
svc_pred = svc_model.predict(X_test)
rf_pred = rf_best_model.predict(X_test)
gb_pred = gb_best_model.predict(X_test)
ada_pred = ada_best_model.predict(X_test)
xgb_pred = xgb_best_model.predict(X_test)
mlp_pred = mlp_best_model.predict(X_test)
extra_trees_pred = extra_trees_best_model.predict(X_test)
knn_pred = knn_best_model.predict(X_test)


# Create new features using base model predictions
X_meta = np.column_stack((rf_pred, gb_pred, ada_pred, xgb_pred, mlp_pred, extra_trees_pred, knn_pred, svc_pred, rf_pred, gb_pred))

meta_model =  RandomForestClassifier()
meta_model.fit(X_meta, y_test)

# Make predictions using the meta-model
y_pred_meta = meta_model.predict(X_meta)

# Evaluation
meta_accuracy = accuracy_score(y_test, y_pred_meta)
print("Meta-Model Accuracy:", meta_accuracy*100)
print(classification_report(y_test, y_pred_meta))

Meta-Model Accuracy: 92.72727272727272
              precision    recall  f1-score   support

           0       0.98      0.95      0.97        44
           1       0.88      0.88      0.88        34
           2       0.91      0.94      0.92        32

    accuracy                           0.93       110
   macro avg       0.92      0.92      0.92       110
weighted avg       0.93      0.93      0.93       110



In [27]:
joblib.dump(meta_model, 'meta_model.joblib')

['meta_model.joblib']

In [28]:
recommendation_models = {}

# Train separate RandomForestClassifier models for each recommendation
for recommendation in recommendations:
    # Train the model
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train_rec, y_train_rec[recommendation])
    recommendation_models[recommendation] = model

# Make predictions on the testing set
y_pred_rec = {recommendation: model.predict(X_test_rec) for recommendation, model in recommendation_models.items()}

# Evaluate the models
accuracy = {recommendation: accuracy_score(y_test_rec[recommendation], y_pred_rec[recommendation]) for recommendation in recommendations}
for recommendation in recommendations:
    print("Accuracy for {}: {:.2f}".format(recommendation, accuracy[recommendation]))

# Calculate overall accuracy
overall_accuracy = sum(accuracy.values()) / len(accuracy)
print("\nOverall Accuracy:", overall_accuracy*100)

Accuracy for Lifestyle Modification: 0.89
Accuracy for T2DM: 0.99
Accuracy for Less salt intake: 0.98
Accuracy for Regular exercise: 0.91
Accuracy for Nutrition rich food: 0.99
Accuracy for Medication: 0.89

Overall Accuracy: 94.24242424242424


In [29]:
for recommendation, model in recommendation_models.items():
    filename = f'{recommendation.replace(" ", "_").lower()}_model.joblib'
    dump(model, filename)
    print(f'Model for {recommendation} saved as {filename}')

Model for Lifestyle Modification saved as lifestyle_modification_model.joblib
Model for T2DM saved as t2dm_model.joblib
Model for Less salt intake saved as less_salt_intake_model.joblib
Model for Regular exercise saved as regular_exercise_model.joblib
Model for Nutrition rich food saved as nutrition_rich_food_model.joblib
Model for Medication saved as medication_model.joblib


In [30]:
user_input = {
    'AGE': 53,
    'MENSTRUAL IRREGULARITY':0,
    'THYROID PROBLEM': 0,
    'WEIGHT': 90,
    'ESTROGEN': 302,
    'FT3': 308,
    'HEART RATE': 92,
    'LH': 15,
    'MOOD SWINGS': 'No',
    'HOT FLASHES': 'No'
}

user_input_encoded = {key: (1 if value == 'Yes' else 0 if value == 'No' else value) for key, value in user_input.items()}

input_df = pd.DataFrame(user_input_encoded, index=[0])

In [31]:
rf_pred_user = rf_best_model.predict([list(user_input_encoded.values())])
gb_pred_user = gb_best_model.predict([list(user_input_encoded.values())])
ada_pred_user = ada_best_model.predict([list(user_input_encoded.values())])
xgb_pred_user = xgb_best_model.predict([list(user_input_encoded.values())])
mlp_pred_user = mlp_best_model.predict([list(user_input_encoded.values())])
extra_trees_pred_user = extra_trees_best_model.predict([list(user_input_encoded.values())])
knn_pred_user = knn_best_model.predict([list(user_input_encoded.values())])
svc_pred_user = svc_model.predict([list(user_input_encoded.values())])
rf_pred_user = rf_best_model.predict([list(user_input_encoded.values())])
gb_pred_user = gb_best_model.predict([list(user_input_encoded.values())])

user_meta_input = np.column_stack((rf_pred_user, gb_pred_user, ada_pred_user, xgb_pred_user, mlp_pred_user, extra_trees_pred_user, knn_pred_user, svc_pred_user,rf_pred_user, gb_pred_user))

c:\Users\Ramya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Ramya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
c:\Users\Ramya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
c:\Users\Ramya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
c:\Users\Ramya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature 

In [32]:
user_pred_meta = meta_model.predict(user_meta_input)
mapping2 = {0: 'Mild', 1: 'Moderate', 2: 'Severe'}
print(f"Predicted severity: {mapping2.get(user_pred_meta[0], 'Unknown')}")

# Make predictions using the trained models
recommendation_pred = {recommendation: model.predict(input_df) for recommendation, model in recommendation_models.items()}

# Print the recommendations
print("Recommendations:")
for recommendation, pred in recommendation_pred.items():
    if pred[0] == 1:
        print(recommendation)

Predicted severity: Severe
Recommendations:
Lifestyle Modification
T2DM
Medication


In [34]:
joblib.dump(rf_best_model, 'rf_model.joblib')


['rf_model.joblib']

In [35]:
joblib.dump(svc_model, 'svc_model.joblib')


['svc_model.joblib']

In [36]:
joblib.dump(meta_model, 'meta_model.joblib')

['meta_model.joblib']

In [39]:
joblib.dump(mlp_best_model, 'mlp_model.joblib')

['mlp_model.joblib']

In [40]:
joblib.dump(gb_best_model, 'gb_model.joblib')

['gb_model.joblib']

In [41]:
joblib.dump(ada_best_model, 'ada_model.joblib')

['ada_model.joblib']

In [42]:
joblib.dump(xgb_best_model, 'xgb_model.joblib')

['xgb_model.joblib']

In [43]:
joblib.dump(extra_trees_best_model, 'et_model.joblib')

['et_model.joblib']

In [44]:
joblib.dump(knn_best_model, 'knn_model.joblib')

['knn_model.joblib']